In [ ]:
# Pin 13=D7 on esp8266
# dallas wiring: flat face towards you, 
# left is ground, right is live, 
# middle is signal with a 4.7k pullup resistor onto the live
# on string white=G, green=Live, red=signal

In [5]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [2]:
%sendtofile --source wifimqttconnections.txt config.txt
%sendtofile --append config.txt

pinled       2
pindallas    13
connection2  BV6000,bubblino,192.168.43.1

Sent 3 lines (72 bytes) to config.txt.


In [3]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 75 lines (2600 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [6]:
# esp32 version
# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py

Sent 646 lines (23109 bytes) to mqtt_as.py.


In [8]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [10]:
%sendtofile main.py

import time
from machine import Pin
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask, flashledconnectedtask
from stdmqttas import pinled, flashpinled, shortmac, topicstem
from mqtt_as import MQTTClient
import onewire, ds18x20

pindallas = Pin(int(fconfig["pindallas"]), Pin.OUT, Pin.PULL_UP)
dallasobj = ds18x20.DS18X20(onewire.OneWire(pindallas))

import network
network.WLAN().active(0)  # disable the connection at startup
flashpinled(5, 300, 300)

def itertools_count():
    n = 0
    while True:
        yield n
        n += 1

topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"
topicerror = topicstem+"/error"
topicdallas = topicstem+'/DS18B20'

async def dallastask():
    btoinit = True
    for i in itertools_count():
        try:
            if btoinit:
                dallasscanned = dallasobj.scan() or []
                sdallasqorder = {347:0, 376:1, 448:2, 475:3, 576:4, 311:5, 406:6, 157:7}
                dallasscanned.sort(key=lambda X: sdallasqorder.get(sum(X), 0))
                print("dallas", dallasscanned)
                btoinit = not dallasscanned
                await asyncio.sleep_ms(100)
            dallasobj.convert_temp()
            await asyncio.sleep_ms(800)
            meas = [ ]
            for d in dallasscanned:
                try:
                    t = dallasobj.read_temp(d)
                except Exception:
                    t = -100
                meas.append(t)
            payload = " ".join(map(str, meas))
            if client.isconnected():
                print(topicdallas, meas)
                await client.publish(topicdallas, payload)
        except OSError as e:
            print("dallastask", e)
            btoinit = True
        await asyncio.sleep_ms(50)
        
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber, "to", client._ssid, client.server)
    await client.publish(topiconline, "1", retain=True)
    await client.publish(topicip, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)

config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topiconline, "0", True)
client = MQTTClient(config)
client.DEBUG = True
    
aloop = asyncio.get_event_loop()
aloop.create_task(dallastask())
aloop.create_task(flashledconnectedtask(client))
aloop.create_task(mqttconnecttask(client))

aloop.run_until_complete(client.publish(topicerror, "", retain=True))
for n in itertools_count():
    try:
        aloop.run_forever()
    except Exception as e:
        aloop.run_until_complete(client.publish(topicerror, str(n)+" "+str(e), retain=True))


Sent 84 lines (2839 bytes) to main.py.
